In [1]:
import polars as pl
import numpy as np
import data_actions.data_processing.utils as utils

In [2]:
data = utils.load_data("data/original_data")
data_cart = data["events"]["add_to_cart"]
data_cart.collect()

c:\Users\Adam\Pulpit\STUDIA\SEM6\SP2\RecSysMain\RecSys2025\data_actions\data_processing\utils.py:67: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "timestamp" in df.columns:


client_id,timestamp,sku
i64,datetime[μs],i64
17649961,2022-08-11 12:22:55,1398840
315805,2022-09-11 05:34:10,1434568
315805,2022-10-06 04:58:15,998226
16696114,2022-07-11 16:27:00,81192
10238779,2022-10-05 14:36:55,1475246
…,…,…
16225940,2022-10-10 16:07:50,498350
3850845,2022-06-12 16:13:59,1461294
12296853,2022-07-29 17:53:45,431834


In [ ]:
from collections import defaultdict

k = 100

# Sort data_cart by timestamp
data_cart_chronological = data_cart.sort("timestamp").collect()

# Mapa klient -> lista sku
client_skus = defaultdict(list)

for row in data_cart_chronological.iter_rows(named=True):
    client_id = row["client_id"]
    sku = row["sku"]
    if len(client_skus[client_id]) < k:
        client_skus[client_id].append(sku)

# Zamień na listę słowników z kolumnami sku_0, sku_1, ..., sku_{k-1}
rows = []
for client_id, skus in client_skus.items():
    row = {"client_id": client_id}
    for i, sku in enumerate(skus):
        row[f"sku_{i}"] = sku
    rows.append(row)

# Utwórz DataFrame
V = pl.DataFrame(rows)
